In [ ]:
# DEPENDENCIES:
# AFNI, FSL, ConnectomeWorkbench, and Freesurfer (the four horsemen)
# R, and R packages "freesurferformats" and "gifti"

In [2]:
cd ~/

### SET SUBJECT:
subj=test
###


### SET DIRECTORIES PROPERLY
### SET DIRECTORIES PROPERLY
### SET DIRECTORIES PROPERLY

FS=~/fs # Freesurfer SUBJECTS_DIR
DOC=~/doc/fs # Separate directory for all files going into freesurfer pipeline
             # Volumes: mask to tessellate, segmentation for labels, brain 
             # Text files: connectome-workbench labels, freesurfer labels=

In [ ]:
# If Freesurfer subject directory must be added:
mksubjdirs $subj

#### 1.a Import volume files

In [74]:
# Setup folders
cd $DOC/$subj
pwd
if [ ! -d "orig" ]; then 'mkdir orig'; fi

/home/mavar/doc/fs/test


In [19]:
# Change headers on all files
cd $DOC/$subj
pwd

for file in *.nii.gz
    do 3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen $file
done

In [88]:
# Check intensity range of files
cd $DOC/$subj
for file in *.nii.gz; do echo "$file $(fslstats ${file} -R)"; done

brain.nii.gz 0.754000 10.159649 
brainmask.nii.gz 0.000000 1.000000 
brainmask_255.nii.gz 0.000000 255.000000 
grey_expand.nii.gz 0.000000 45.000000 
wmfix-test.nii.gz 0.000000 255.000000 
wmfix.nii.gz 0.000000 75593097302243013842075385856.000000 


In [67]:
# Change intensity range of masks, if needed

### Selected file:
file=wmfix.nii.gz
###

cp ${file} orig/beforemult_${file}
fslmaths $file -mul 255  $file

In [ ]:
# DO NOT RUN!!!
# Attempt to use regex's to automatically set the file intensities. This is dangerous, and unnecessary!

# for maskfile in *mask.nii.gz
# do mask_range=`fslstats ${maskfile} -R` # get range of 
# mask_upper_bound=$(tmp=${mask_range#* }; echo ${tmp%.*}) # regex to get white matter upper bound
# if [$mask_upper_bound -e 255]
#     then
#         echo "yes";
# else
#         fslmaths ${maskfile} -mul $(echo 255 / $mask_upper_bound | bc -l) ${maskfile%%.*}_255.nii.gz # multiply mask by upper bound
# fi
# done

In [ ]:
# Convert files to Freesurfer format and move to ~/fs/
cd $DOC/$subj
for file in *.nii.gz; do mri_convert $file ${FS}/${subj}/mri/${file%%.*}.mgz; done

#### 1.b Prepare surface

In [ ]:
cd $FS/$subj
pwd

#
# Select volume to tessellate for flattening..
# You must do some repairs before flattening
#

### Selected volume:
volume=wmfix.mgz
###

mri_tessellate mri/${volume} 1 surf/lh.orig.nofix # arbitary intensity value 255 is set above with fslmaths

# Recommended to view the tessellation before committing to post-processing
# freeview -f lh.orig.nofix

In [ ]:
# Post-process tessellation

cd ${FS}/$subj/surf
pwd
echo "This will take a while."

mris_extract_main_component  lh.orig.nofix lh.orig.nofix
mris_smooth -nw lh.orig.nofix lh.smoothwm.nofix
mris_inflate lh.smoothwm.nofix lh.inflated.nofix
mris_sphere -q lh.inflated.nofix lh.qsphere.nofix

# attempt to automatically fix topology
mris_euler_number ${fsSurf_temp}/${xh}.orig
mris_remove_intersection lh.orig lh.orig
mris_smooth lh.orig lh.smoothwm
mris_inflate lh.smoothwm lh.inflated

#### 2--Import labels

In [103]:
# Prep folder

cd ${FS}/${subj}/surf
if [ ! -d "label-workspace" ]; then `mkdir label-workspace`; echo "created folder"; fi
cd label-workspace
pwd

/home/mavar/fs/test/surf/label-workspace


In [115]:
# Grab label file
# Assumed it is in ~/doc/fs/

### Label file name:
labelfile=grey_expand.nii.gz
###

cp ${DOC}/${subj}/${labelfile} ${FS}/${subj}/surf/label-workspace/${labelfile}
pwd
ls ${FS}/${subj}/surf/label-workspace/

/home/mavar/fs/test/surf/label-workspace
grey.label.nii  grey_expand.nii.gz  labelreal.txt  lh.label.gii  lh.surf.gii


In [107]:
# Grab connectome-workbench label list file
# You need to generate it in the connectome-workbench format
# (2 lines per label)
# <labelname>
# <key> <red> <green> <blue> <alpha>
# Per https://www.humanconnectome.org/software/workbench-command/-volume-label-import

# <key> is a number 1..(number of labels). It's recommended that <labelname> also just be a number
# the colors don't matter, as they're assigned by the freesurfer color table

### List file name:
listfile=labelreal.txt
###

cp ${DOC}/${subj}/${listfile} ${FS}/${subj}/surf/label-workspace/${listfile}

In [5]:
# Grab freesurfer color table 
# Happily, this uses the same format as Slicer

### Color table name:
colortable=color.txt
###

cp ${DOC}/${subj}/${colortable} ${FS}/${subj}/label/${colortable}

ls ${FS}/${subj}/label

color.txt


In [108]:
# Generate connectome-workbench "volume-label" file
# This output format is NII, not gii. Naming convention *.label.nii

wb_command -volume-label-import ${labelfile} labelreal.txt grey.label.nii

In [3]:
# Grab surface to label
# Obviously, this needs to be in the same space as the volume labels, so we use the original surface generated from the segmentation
# The output format is *.surf.gii

cd ${FS}/${subj}/surf

mris_convert --to-scanner lh.orig.nofix label-workspace/lh.surf.gii
# --to-tkr flag is needed for output surface to oriented properly

cd label-workspace
pwd

Converting from tkr to scanner coordinates
Saving label-workspace/lh.surf.gii as a surface
/home/mavar/fs/test/surf/label-workspace


In [4]:
# Generate connectome-workbench "surface-label" file
# The format is *.label.gii

wb_command -volume-label-to-surface-mapping grey.label.nii lh.surf.gii lh.label.gii
ls

1.label         grey_expand.nii.gz  lh.label.gii  logs
grey.label.nii  labelreal.txt       lh.surf.gii   output


In [ ]:
# Import the surface label to freesurfer
# This is the annoying part

### Number of segments in label file:
nsegments=45
###

# Use the R library "freesurferformats"
# Read the vertex indices of each label as an integer vector, then write them back to a new label file

echo "library(\"freesurferformats\")
options(scipen=999)
for (i in 1:${nsegments}) {write.fs.label(sprintf(\"/home/mavar/fs/test/surf/label-workspace/output/%d.label\",i),read.fs.label.gii(\"/home/mavar/fs/test/surf/label-workspace/lh.label.gii\",i))}" \
| R --no-save # Echo that string to an R prompt

In [6]:
# Collect labels in Freesurfer .annot file
# Interesting musings on the FS label/annotation system:
# https://surfer.nmr.mgh.harvard.edu/fswiki/LabelsClutsAnnotationFiles

labelregex=''
for i in $(seq 1 $nsegments); do labelregex="$labelregex --l ${FS}/${subj}/surf/label-workspace/output/${i}.label"; done

# I used ChatGPT to make this string before, as I was lazy

rm ${FS}/${subj}/label/lh.grey.annot
# This is the only function in Freesurfer than warns you before overwriting files (which I actually like)

# The documentation on this is particularly poor; the man page for mris_label2annot does not even specify what the required arguments of the function are
mris_label2annot --s $subj --hemi lh --ctab ${FS}/${subj}/label/${colortable} ${labelregex} --a grey

rm: cannot remove '/home/mavar/fs/test/label/lh.grey.annot': No such file or directory
Reading ctab /home/mavar/fs/test/label/color.txt
Number of ctab entries 46

7.3.2
cwd /home/mavar
cmdline mris_label2annot --s test --hemi lh --ctab /home/mavar/fs/test/label/color.txt --l /home/mavar/fs/test/surf/label-workspace/output/1.label --l /home/mavar/fs/test/surf/label-workspace/output/2.label --l /home/mavar/fs/test/surf/label-workspace/output/3.label --l /home/mavar/fs/test/surf/label-workspace/output/4.label --l /home/mavar/fs/test/surf/label-workspace/output/5.label --l /home/mavar/fs/test/surf/label-workspace/output/6.label --l /home/mavar/fs/test/surf/label-workspace/output/7.label --l /home/mavar/fs/test/surf/label-workspace/output/8.label --l /home/mavar/fs/test/surf/label-workspace/output/9.label --l /home/mavar/fs/test/surf/label-workspace/output/10.label --l /home/mavar/fs/test/surf/label-workspace/output/11.label --l /home/mavar/fs/test/surf/label-workspace/output/12.label --l /

In [ ]:
# Test it out

freeview -f lh.orig annot=lh.grey.annot

In [ ]:
# Now you're on your own. Cut a patch in Freeview, run mris_flatten and pray.